<a href="https://colab.research.google.com/github/isamdr86/ai-project/blob/main/notebooks/Web_Search_API_ir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index==0.10.57 openai==1.37.0 tiktoken==0.7.0 llama-index-tools-google==0.1.3 newspaper4k==0.9.3.1 lxml_html_clean

In [2]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [21]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

GOOGLE_SEARCH_KEY = userdata.get('google_search_api_key')
GOOGLE_SEARCH_ENGINE = userdata.get('google_engine_id') # Search Engine ID

## LLM and Embedding Model

In [2]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model="gpt-4o-mini", temperature= 0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Using Agents/Tools


In [3]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent


# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

# initialize ReAct agent
agent = ReActAgent.from_tools([multiply_tool], verbose=True)

In [4]:
res = agent.chat("What is the multiplication of 43 and 45?")

# Final response
res.response

> Running step 2a90f31f-9dfc-4847-93c8-f656c435c5c8. Step input: What is the multiplication of 43 and 45?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 43, 'b': 45}
Observation: 1935
> Running step 0087f48b-c3ab-4b22-b0ca-5f7d67e0ff1c. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The multiplication of 43 and 45 is 1935.


'The multiplication of 43 and 45 is 1935.'

## Define Google Search Tool


In [5]:
from llama_index.tools.google import GoogleSearchToolSpec

tool_spec = GoogleSearchToolSpec(key=GOOGLE_SEARCH_KEY, engine=GOOGLE_SEARCH_ENGINE)

In [6]:
# Import and initialize our tool spec
from llama_index.core.tools.tool_spec.load_and_search import LoadAndSearchToolSpec

# Wrap the google search tool to create an index on top of the returned Google search
wrapped_tool = LoadAndSearchToolSpec.from_defaults(
    tool_spec.to_tool_list()[0],
).to_tool_list()

## Create the Agent


In [7]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(wrapped_tool, verbose=False)

In [8]:
res = agent.chat("How many parameters LLaMA 3.2 model has?")

In [9]:
res.response

'The LLaMA 3.2 model has the following parameters:\n\n- Vision models: 11 billion and 90 billion parameters\n- Text-only models: 1 billion and 3 billion parameters'

In [10]:
res.sources

[ToolOutput(content='Content loaded! You can now search the information using read_google_search', tool_name='google_search', raw_input={'args': (), 'kwargs': {'query': 'LLaMA 3.2 model parameters'}}, raw_output='Content loaded! You can now search the information using read_google_search', is_error=False),
 ToolOutput(content='Llama 3.2 features small and medium-sized vision models with 11 billion and 90 billion parameters, alongside lightweight text-only models with 1 billion and 3 billion parameters. It also introduces the Llama Stack Distribution.', tool_name='read_google_search', raw_input={'args': (), 'kwargs': {'query': 'LLaMA 3.2 model parameters'}}, raw_output='Llama 3.2 features small and medium-sized vision models with 11 billion and 90 billion parameters, alongside lightweight text-only models with 1 billion and 3 billion parameters. It also introduces the Llama Stack Distribution.', is_error=False)]

# Using Tools w/ VectorStoreIndex


A limitation of the current agent/tool in LlamaIndex is that it **relies solely on the page description from the retrieved pages** to answer questions. This approach will miss answers that are not visible in the page's description tag. To address this, a possible workaround is to fetch the page results, extract the page content using the newspaper3k library, and then create an index based on the downloaded content. Also, the previous method stacks all retrieved items from the search engine into a single document, making it **difficult to pinpoint the exact source** of the response. However, the following method will enable us to present the sources easily.


## Define Google Search Tool


In [11]:
from llama_index.tools.google import GoogleSearchToolSpec

tool_spec = GoogleSearchToolSpec(key=GOOGLE_SEARCH_KEY, engine=GOOGLE_SEARCH_ENGINE)

In [12]:
search_results = tool_spec.google_search("LLaMA 3.2 model details")

In [13]:
import json

search_results = json.loads(search_results[0].text)

In [14]:
search_results

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - LLaMA 3.2 model details',
    'totalResults': '6620000',
    'searchTerms': 'LLaMA 3.2 model details',
    'count': 10,
    'startIndex': 1,
    'inputEncoding': 'utf8',
    'outputEncoding': 'utf8

## Read Each URL Contents


In [15]:
import newspaper

pages_content = []

for item in search_results["items"]:

    try:
        article = newspaper.Article(item["link"])
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append(
                {"url": item["link"], "text": article.text, "title": item["title"]}
            )
    except:
        continue

print(len(pages_content))

6


## Create the Index


In [16]:
from llama_index.core import Document

# Convert the texts to Document objects so the LlamaIndex framework can process them.
documents = [
    Document(text=row["text"], metadata={"title": row["title"], "url": row["url"]})
    for row in pages_content
]

In [17]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

# Build index / generate embeddings using OpenAI.
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[SentenceSplitter(chunk_size=512, chunk_overlap=128)],
)

In [18]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = index.as_query_engine()

## Query


In [19]:
response = query_engine.query("How many parameters LLaMA 3.2 model has? list exact sizes of the models")
print(response)

The LLaMA 3.2 model has variants with the following exact sizes in terms of parameters:

- 1B (1 billion parameters)
- 3B (3 billion parameters)
- 90B (90 billion parameters)


In [20]:
# Show the retrieved nodes
for src in response.source_nodes:
    print("Title\t", src.metadata["title"])
    print("Source\t", src.metadata["url"])
    print("Score\t", src.score)
    print("-_" * 20)

Title	 Llama 3.2 Requirements [What you Need to Use It?]
Source	 https://llamaimodel.com/requirements-3-2/
Score	 0.5604090406471472
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Title	 Llama 3.2 Requirements [What you Need to Use It?]
Source	 https://llamaimodel.com/requirements-3-2/
Score	 0.5404580362425195
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
